In [ ]:
!pip install git+https://github.com/PennyLaneAI/pennylane/@master
!pip install git+https://github.com/QSAR-UBC/ionizer@main
!pip install qiskit
!pip install pennylane-qiskit

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import Identity, PauliX, PauliY, PauliZ

import ionizer
from ionizer.transforms import ionize, commute_through_ms_gates, single_qubit_fusion_gpi

In [2]:
coeffs = np.array([
    -10.943495,   6.590075,   6.59009 ,   6.59008 ,   6.1535  ,
    -3.991875,  -3.991875,  -0.56607 ,  -0.400535,  -0.334915,
    -0.841875,  -0.34829 ,  -0.665395,  -0.33492 ,  -0.545725,
    -0.644445,  -0.334915,  -0.39765 ,  -0.792515,  -0.282125,
    -0.282125,  -0.225   ,   0.34782 ,   0.34782 ,  -0.182285,
    -0.182285,  -0.419445,  -0.419445,  -0.667545,  -0.667545,
    -0.24693 ,   0.24693 ,   0.44714 ,   0.44714 ,   0.419445,
     0.419445,   0.415175,   0.415175,   0.4993  ,  -0.4993  ,
    -0.419445,  -0.419445,  -0.28899 ,  -0.28899 ,  -0.625485,
     0.625485,  -0.22309 ,  -0.22309 ,  -0.22309 ,   0.22309 ,
    -0.27825 ,   0.27825 
])

ops = [
    Identity(wires=[0]),
    PauliZ(wires=[0]),
    PauliZ(wires=[1]),
    PauliZ(wires=[2]),
    PauliZ(wires=[3]),
    PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]),
    PauliZ(wires=[0]) @ PauliZ(wires=[2]),
    PauliZ(wires=[0]) @ PauliZ(wires=[3]),
    PauliZ(wires=[0]) @ PauliZ(wires=[4]),
    PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[1]) @ PauliZ(wires=[2]),
    PauliZ(wires=[1]) @ PauliZ(wires=[3]),
    PauliZ(wires=[1]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[2]) @ PauliZ(wires=[3]),
    PauliZ(wires=[2]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]),
    PauliY(wires=[0]) @ PauliY(wires=[1]),
    PauliX(wires=[0]) @ PauliX(wires=[1]),
    PauliY(wires=[0]) @ PauliY(wires=[2]),
    PauliX(wires=[0]) @ PauliX(wires=[2]),
    PauliY(wires=[0]) @ PauliY(wires=[3]),
    PauliX(wires=[0]) @ PauliX(wires=[3]),
    PauliY(wires=[0]) @ PauliY(wires=[4]),
    PauliX(wires=[0]) @ PauliX(wires=[4]),
    PauliX(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[0]),
    PauliY(wires=[1]) @ PauliY(wires=[2]),
    PauliX(wires=[1]) @ PauliX(wires=[2]),
    PauliY(wires=[1]) @ PauliY(wires=[3]),
    PauliX(wires=[1]) @ PauliX(wires=[3]),
    PauliY(wires=[1]) @ PauliY(wires=[4]),
    PauliX(wires=[1]) @ PauliX(wires=[4]),
    PauliZ(wires=[0]) @ PauliX(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[1]),
    PauliY(wires=[2]) @ PauliY(wires=[3]),
    PauliX(wires=[2]) @ PauliX(wires=[3]),
    PauliY(wires=[2]) @ PauliY(wires=[4]),
    PauliX(wires=[2]) @ PauliX(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliX(wires=[2]) @ PauliZ(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[2]),
    PauliY(wires=[3]) @ PauliY(wires=[4]),
    PauliX(wires=[3]) @ PauliX(wires=[4]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliX(wires=[3]) @ PauliZ(wires=[4]),
    PauliX(wires=[3]),
    PauliZ(wires=[0]) @ PauliZ(wires=[1]) @ PauliZ(wires=[2]) @ PauliZ(wires=[3]) @ PauliX(wires=[4]),
    PauliX(wires=[4])
]

In [3]:
H_tapered = qml.Hamiltonian(coeffs, ops)

hf_state_tapered = np.array([1, 1, 0, 0, 0])

In [4]:
def exp_decomposition(param, y_wire, y_wire_sign=1):
    """Applies a compact version of Exp("Z..ZYZ..Z") for some parameter, 
    where we pass the index of the Y wire."""
    qml.RZ(np.pi/2, wires=y_wire)
    qml.RY(np.pi/2, wires=y_wire)
    qml.RZ(-np.pi/2, wires=y_wire)
    qml.CNOT(wires=[4, 3])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[1, 0])
    qml.RZ(param, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[3, 1])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[4, 3])
    qml.RZ(y_wire_sign * param, wires=y_wire)
    qml.RZ(-np.pi/2, wires=y_wire)
    qml.RY(np.pi/2, wires=y_wire)
    qml.RZ(np.pi/2, wires=y_wire)   


@qml.qfunc_transform
def expand_rot_and_remove_zeros(tape):
    for op in tape:
        if op.name == "Rot":
            if not np.isclose(op.data[0], 0.0):
                qml.RZ(op.data[0], wires=op.wires)
            if not np.isclose(op.data[1], 0.0):
                qml.RY(op.data[1], wires=op.wires)
            if not np.isclose(op.data[2], 0.0):
                qml.RZ(op.data[2], wires=op.wires)
        else:
            qml.apply(op)


@qml.qfunc_transform
def cnot_folding(tape, scale_factor):
    num_pairs = qml.math.round((scale_factor - 1.0) / 2.0)

    for op in tape:
        qml.apply(op)

        if op.name == "CNOT":
            for _ in range(int(2 * num_pairs)):
                qml.apply(op)


In [5]:
dev = qml.device("default.qubit", wires=range(len(H_tapered.wires)))

@qml.qnode(dev)
@expand_rot_and_remove_zeros
@qml.transforms.single_qubit_fusion()
def tapered_circuit_simplified(params):
    for wire in dev.wires:
        if hf_state_tapered[wire] == 1:
            qml.PauliX(wires=wire)
    qml.RY(params[0], wires=2)
    qml.CNOT(wires=[2, 0])
    qml.SingleExcitation.compute_decomposition(params[1], wires=[0, 3])
    qml.SingleExcitation.compute_decomposition(params[2], wires=[0, 4])
    exp_decomposition(params[3], y_wire=0, y_wire_sign=-1)
    qml.SingleExcitation.compute_decomposition(params[4], wires=[1, 2])
    qml.SingleExcitation.compute_decomposition(params[5], wires=[1, 3])
    qml.SingleExcitation.compute_decomposition(params[6], wires=[1, 4])
    exp_decomposition(params[7], y_wire=1, y_wire_sign=-1)
    return qml.expval(H_tapered)

In [6]:
params = np.array([ 1.20436673, -1.13296563, -0.16546356, -0.06453663,  1.0023775 ,
         1.29885146,  0.12871688,  0.0831842 ], requires_grad=True)

In [7]:
print(qml.draw(tapered_circuit_simplified)(params))

0: ──RZ(1.57)──RY(3.14)──RZ(-1.57)─╭X──RZ(2.36)──RY(1.57)──RZ(1.57)─╭X──RZ(0.57)──╭X──────────
1: ──RZ(1.57)──RY(3.14)──RZ(-1.57)─│────────────────────────────────│─────────────│───────────
2: ──RY(1.20)──────────────────────╰●───────────────────────────────│─────────────│───────────
3: ──RZ(0.79)──RY(1.57)─────────────────────────────────────────────╰●──RY(-0.57)─╰●──RZ(3.14)
4: ──RZ(0.79)──RY(1.57)───────────────────────────────────────────────────────────────────────

──────────────────────╭X──RZ(0.08)──╭X──RZ(-3.14)──RY(0.79)─────────────────╭X──RZ(-0.06)─╭X
──────────────────────│─────────────│──╭X────────────────────────────────╭X─╰●────────────╰●
──────────────────────│─────────────│──╰●────────────────────────────────│──────────────────
───RY(1.57)──RZ(2.36)─│─────────────│─────────────────────────────────╭X─╰●─────────────────
──────────────────────╰●──RY(-0.08)─╰●──RZ(3.14)───RY(1.57)──RZ(2.36)─╰●────────────────────

───RZ(-1.51)──RY(1.57)──RZ(1.57)──────────────────────────

## Convert to trapped ion gates

In [8]:
from qiskit import QuantumCircuit
from qiskit import transpile

# Get the tape and run it through the tranaspiler; use the optimal
# params but convert to non-diff version
original_tape = tapered_circuit_simplified.qtape
original_qasm = original_tape.to_openqasm()

In [9]:
qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

In [10]:
# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

@qml.qnode(dev)
@cnot_folding(scale_factor=3)
def tapered_circuit_transpiled():
    new_qfunc()
    qml.RZ(np.pi/2, wires=1)
    return qml.expval(H_tapered)
                
print(np.round(tapered_circuit_transpiled(), decimals=6))

-21.483662


/home/jovyan/.local/lib/python3.9/site-packages/pennylane_qiskit/converter.py:227: UserWarning: pennylane_qiskit.converter: The Measure instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(


In [11]:
qml.specs(tapered_circuit_transpiled)()

/home/jovyan/.local/lib/python3.9/site-packages/pennylane/tape/qscript.py:1386: UserWarning: The gate_sizes key is deprecated and will be removed in the next release. Going forward, please use: qml.specs()["resources"].gate_sizes
  warnings.warn(
/home/jovyan/.local/lib/python3.9/site-packages/pennylane/tape/qscript.py:1386: UserWarning: The gate_types key is deprecated and will be removed in the next release. Going forward, please use: qml.specs()["resources"].gate_types
  warnings.warn(


{'resources': Resources(num_wires=5, num_gates=133, gate_types=defaultdict(<class 'int'>, {'RY': 26, 'RZ': 29, 'CNOT': 78}), gate_sizes=defaultdict(<class 'int'>, {1: 55, 2: 78}), depth=89, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 55, 2: 78}),
 'gate_types': defaultdict(int, {'RY': 26, 'RZ': 29, 'CNOT': 78}),
 'num_operations': 133,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 52,
 'depth': 89,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

In [12]:
@qml.qnode(dev)
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="right")
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="left")
@single_qubit_fusion_gpi
@ionizer.transforms.commute_through_ms_gates(direction="left")
@ionize
@cnot_folding(scale_factor=3)
def tapered_circuit_ionized(params):
    new_qfunc()
    qml.RZ(np.pi/2, wires=1) 
    return qml.expval(H_tapered)
                      
print(np.round(tapered_circuit_ionized(params), decimals=6))

-21.483662


In [13]:
print(qml.draw(tapered_circuit_ionized)(params))

0: ──────────────────────────────────────╭MS─╭MS─╭MS──GPI2(-1.57)──GPI(-1.96)──GPI2(2.36)
1: ──────────────────────────────────────│───│───│───╭MS──────────╭MS─────────╭MS────────
2: ──GPI2(1.57)──GPI(0.60)───GPI2(-1.94)─╰MS─╰MS─╰MS─╰MS──────────╰MS─────────╰MS────────
3: ──GPI2(0.23)──GPI(-1.96)──GPI2(-1.01)─────────────────────────────────────────────────
4: ──GPI2(0.23)──GPI(-1.96)──GPI2(-1.01)─────────────────────────────────────────────────

───────────────────────────╭MS─╭MS─╭MS──GPI(-0.28)─╭MS─╭MS─╭MS────────────────────────────────────
───────────────────────────│───│───│───────────────│───│───│──────────────────────────────────────
───GPI2(-1.23)──GPI(-1.40)─│───│───│───────────────│───│───│──────────────────────────────────────
───────────────────────────╰MS─╰MS─╰MS──GPI(-0.28)─╰MS─╰MS─╰MS──GPI2(0.79)──GPI(0.39)──GPI2(-1.57)
──────────────────────────────────────────────────────────────────────────────────────────────────

──╭MS─╭MS─╭MS──GPI(-0.04)─╭MS─╭MS─╭MS──GPI(-3.14)───G

In [14]:
qml.specs(tapered_circuit_ionized)(params)

{'resources': Resources(num_wires=5, num_gates=159, gate_types=defaultdict(<class 'int'>, {'GPI2': 44, 'GPI': 37, 'MS': 78}), gate_sizes=defaultdict(<class 'int'>, {1: 81, 2: 78}), depth=104, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 81, 2: 78}),
 'gate_types': defaultdict(int, {'GPI2': 44, 'GPI': 37, 'MS': 78}),
 'num_operations': 159,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 5,
 'num_trainable_params': 52,
 'depth': 104,
 'num_device_wires': 5,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Split into commuting sets

In [15]:
H_tapered_grouped = qml.Hamiltonian(H_tapered.coeffs, H_tapered.ops, grouping_type="qwc")
ham_tapes, ham_fn = qml.transforms.hamiltonian_expand(tapered_circuit_ionized.qtape, group=True)

In [16]:
ham_tapes = [ionize.tape_fn(tape) for tape in ham_tapes]

In [17]:
res = qml.execute(ham_tapes, dev)

In [18]:
ham_fn(res)

array(-21.48366217)

In [1]:
from azure.quantum.cirq import AzureQuantumService
from azure.quantum import Workspace

service = AzureQuantumService(
   resource_id = "",
   location = "westus2"
)

workspace = Workspace(
   resource_id = "",
   location = "westus2"
)

In [20]:
target, target_string = workspace.get_targets(name="ionq.qpu.aria-1"), "HW"
# target, target_string = workspace.get_targets(name="ionq.simulator"), "SIM"

In [21]:
shots = 1000
isotope = 20

job_circuits = [ionizer.utils.tape_to_json(tape, f"HW-O{isotope}-{target_string}-{idx}", shots=shots) for idx, tape in enumerate(ham_tapes)]

In [22]:
for idx, circuit in enumerate(job_circuits):
    cost = target.estimate_cost(circuit["body"], num_shots=shots)
    print(f"Estimated cost of circuit {idx}: {cost.estimated_total} ")

Estimated cost of circuit 0: 35.28 
Estimated cost of circuit 1: 35.0595 
Estimated cost of circuit 2: 35.500499999999995 
Estimated cost of circuit 3: 35.0595 
Estimated cost of circuit 4: 35.28 
Estimated cost of circuit 5: 35.500499999999995 
Estimated cost of circuit 6: 35.721 
Estimated cost of circuit 7: 36.3825 


In [ ]:
jobs = []

for idx, circuit in enumerate(job_circuits):
    jobs.append(target.submit(circuit["body"], name=f"O{isotope}-{target_string}-{idx}", num_shots=shots))
    print(jobs[-1].id)

In [2]:
job_ids = [
    ""
]

jobs = [workspace.get_job(id) for id in job_ids]

In [3]:
job_results = []
for job in jobs:
    job_results.append(job.get_results())

In [ ]:
job_results

In [25]:
def eigenvalue(pauli, bitstring):
    """Given a Pauli word and a bitstring, return whether it is a +1 or -1 
    eigenvalue."""
    eigval = 1
    
    pauli_string = qml.pauli.pauli_word_to_string(pauli, wire_map={i: i for i in range(len(dev.wires))})

    for bit, p in zip(bitstring, pauli_string):
        if bit == "1" and p != "I":
            eigval *= -1
    
    return eigval

In [26]:
def hist_to_counts(hist, num_shots):
    return {np.binary_repr(int(key), 5)[::-1]: int(np.round(num_shots * val)) for key, val in list(hist.items())}


In [27]:
sim_results = [hist_to_counts(result['histogram'], num_shots=shots) for result in job_results]

In [28]:
processed_results = []

for idx, results in enumerate(sim_results):
    results_this_circuit = []
    
    # Get the Hamiltonian terms measured in this case
    ham_terms = [H_tapered_grouped.ops[op_idx] for op_idx in H_tapered_grouped.grouping_indices[idx]]
    
    for term in list(ham_terms):
        expval = 0.0
        
        for bitstring, count in list(results.items()):
            sign = eigenvalue(term, bitstring)
            expval += sign * count
        results_this_circuit.append(expval / shots)
            
    processed_results.append(np.array(results_this_circuit))
    
processed_results = np.array(processed_results)

/home/jovyan/.local/lib/python3.9/site-packages/autograd/numpy/numpy_wrapper.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return _np.array(args, *array_args, **array_kwargs)


In [29]:
processed_results

tensor([tensor([ 1.   ,  0.028,  0.014, -0.114,  0.694,  0.01 , -0.17 , -0.154,
                 0.038, -0.104,  0.008, -0.108,  0.004,  0.05 ], requires_grad=True),
        tensor([ 0.148, -0.168, -0.04 ,  0.14 , -0.016,  0.09 , -0.022], requires_grad=True),
        tensor([ 0.092, -0.066,  0.042, -0.098], requires_grad=True),
        tensor([ 0.272, -0.202,  0.264], requires_grad=True),
        tensor([-0.02,  0.01], requires_grad=True),
        tensor([-0.062,  0.03 ], requires_grad=True),
        tensor([-0.14 ,  0.278,  0.316,  0.132, -0.39 , -0.088, -0.132,  0.138,
                 0.066,  0.172], requires_grad=True)                           ,
        tensor([-0.346, -0.308,  0.04 ,  0.162,  0.002,  0.056, -0.114,  0.144,
                 0.234,  0.034], requires_grad=True)                           ],
       dtype=object, requires_grad=True)

In [30]:
ham_fn(processed_results)

tensor(-15.08974157, requires_grad=True)